# Weak and Nonbinder similar

We are interested to explore the results in a new task type, which is similar to vs Weak and vs Nonbinder.

We will keep in the negative datasets only the sequences that are very close to the positive dataset. In such way, we will generate vs Weak_similar and vs Nonbinder_similar. How similar? This we'll explore in this notebook. Based on the analysis from Aygul, we should be able to find a lot of sequences that are 1-mutation away from positive dataset.

Implementationwise, I think the easiest way will be to define new antigens (e.g. 1ADQSIM) in MiniAbsolut, where we will edit the negative sets. We will later redefine these results as a separate tasks in the following analysis notebooks. We could of course implement a new task type, which would be more robust, but requires more effort into the codebase, which I think is unjustified for a small side-analysis. Later we can reconsider.

To evaluate similar seqeunces and to implement, we first run, for each antigen, CompAIRR between high and weak and between high and nonbinders. We will then analyse the results and using the sequence pairs (separated by 1 edit distance) we will construct the new antigens in which high and weak/nonbinder are either similar or dissimilar.

First, we need to generate the tsvs that can serve as input to CompAIRR, then we run CompAIRR. This was already done by us in sript_04*, we utilise the code from there.

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import shutil

from NegativeClassOptimization import ml
from NegativeClassOptimization import utils, config
from NegativeClassOptimization import preprocessing

/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
from utils_07 import load_trainrest_from_miniabsolut

In [3]:
## UNCOMMENT TO RUN


# for ag in config.ANTIGENS:

#     print(ag)

#     # Load train and rest data for each task type
#     # from MiniAbsolut
#     df = load_trainrest_from_miniabsolut(ag)
#     df["UID"] = df["binder_type"] + "__" + df["ID_slide_Variant"]

#     # Generate df compatible with CompAIRR as input
#     df_AIRR = df.copy()
#     column_map = {
#             "binder_type": "repertoire_id",
#             "UID": "sequence_id",
#             "Slide": "junction_aa",
#         }
#     df_AIRR = df[list(column_map.keys())].copy()
#     df_AIRR = df_AIRR.rename(columns=column_map)

#     # Write
#     out_dir = config.DATA_BASE_PATH / f"MiniAbsolut/{ag}" / "AIRR"
#     out_dir.mkdir(exist_ok=True)
#     out_data_dir = out_dir / "data"
#     out_data_dir.mkdir(exist_ok=True)
#     df_AIRR.to_csv(out_data_dir / "AIRR.tsv", sep='\t', index=False)
#     print(f"Written to {out_data_dir / 'AIRR.tsv'}")

In [4]:
%%bash

## UNCOMMENT TO RUN

## Adapted from script_04b to make it run in a Jupyter notebook cell

# AG='1ADQ'

# # Run for each AG
# for AG in '3VRL' '1NSN' '3RAJ' '5E94' '1H0D' '1WEJ' '1ADQ' '1FBI' '2YPV' '1OB1'
# do
#     echo ${AG}

#     AG_PATH=${PWD}/../data/MiniAbsolut/${AG}/AIRR
#     AIRR_FILE_RELATIVE_PATH=data/AIRR.tsv

#     docker run -v ${AG_PATH}:/opt/compairr torognes/compairr \
#         --ignore-genes --ignore-counts \
#         --threads 12 \
#         --matrix \
#         --differences 1 \
#         --log overlaps_d1.log \
#         --output overlaps_d1_output.tsv \
#         --pairs overlaps_d1_pairs.tsv \
#         ${AIRR_FILE_RELATIVE_PATH} ${AIRR_FILE_RELATIVE_PATH}
    
#     echo "Done for ${AG}"
# done



Let's analyse the results, to evaluate how many sequences are 1-mutation away from the positive dataset. We have the results in an output file, so we want to compare with the file displaying the pairs, and make sure we understand the numbers.

In [5]:
fp = Path("01d_df_res.tsv")

if fp.exists():
    df_res = pd.read_csv(fp, sep='\t')
else:
    records = []
    for ag in config.ANTIGENS:

        print(ag)

        df_pairs  = pd.read_csv(
            f"../data/MiniAbsolut/{ag}/AIRR/overlaps_d1_pairs.tsv",
            sep='\t',
        )
        df_pairs = df_pairs.rename(columns={"#repertoire_id_1": "repertoire_id_1"})
        # print(df_pairs.shape)
        # df_pairs.head()

        df_hw = df_pairs.query(f"repertoire_id_1 == '{ag}_high' and repertoire_id_2 == '{ag}_looserX'")
        num_seq_h_sim_to_w = len(df_hw["sequence_id_1"].unique())
        num_seq_w_sim_to_h = len(df_hw["sequence_id_2"].unique())

        df_hn = df_pairs.query(f"repertoire_id_1 == '{ag}_high' and repertoire_id_2 == '{ag}_95low'")
        num_seq_h_sim_to_n = len(df_hn["sequence_id_1"].unique())
        num_seq_n_sim_to_h = len(df_hn["sequence_id_2"].unique())

        df_miniabs = load_trainrest_from_miniabsolut(ag)
        num_h_total = sum(df_miniabs["binder_type"] == f'{ag}_high')
        num_w_total = sum(df_miniabs["binder_type"] == f'{ag}_looserX')
        num_n_total = sum(df_miniabs["binder_type"] == f'{ag}_95low')

        records.append({
            "antigen": ag,
            "num_seq_h_sim_to_w": num_seq_h_sim_to_w,
            "num_seq_w_sim_to_h": num_seq_w_sim_to_h,
            "num_seq_h_sim_to_n": num_seq_h_sim_to_n,
            "num_seq_n_sim_to_h": num_seq_n_sim_to_h,
            "num_h_total": num_h_total,
            "num_w_total": num_w_total,
            "num_n_total": num_n_total,
        })

    df_res = pd.DataFrame(records)
    df_res.to_csv(fp, sep='\t')

In [6]:
df_res["perc_h_sim_to_w"] = df_res["num_seq_h_sim_to_w"] / df_res["num_h_total"]
df_res["perc_w_sim_to_h"] = df_res["num_seq_w_sim_to_h"] / df_res["num_w_total"]
df_res["perc_h_sim_to_n"] = df_res["num_seq_h_sim_to_n"] / df_res["num_h_total"]
df_res["perc_n_sim_to_h"] = df_res["num_seq_n_sim_to_h"] / df_res["num_n_total"]

df_res

,Unnamed: 0,antigen,num_seq_h_sim_to_w,num_seq_w_sim_to_h,num_seq_h_sim_to_n,num_seq_n_sim_to_h,num_h_total,num_w_total,num_n_total,perc_h_sim_to_w,perc_w_sim_to_h,perc_h_sim_to_n,perc_n_sim_to_h
0,0,3VRL,14093,33160,3271,3748,24456,162506,353663,0.576259,0.204054,0.133750,0.010598
1,1,1NSN,10929,19679,4222,4316,25517,206923,403671,0.428303,0.095103,0.165458,0.010692
2,2,3RAJ,8202,21767,1721,2142,17197,227120,379685,0.476944,0.095839,0.100076,0.005642
3,3,5E94,10264,28090,1644,1942,17016,247905,389535,0.603197,0.113310,0.096615,0.004985
4,4,1H0D,13858,29319,4426,4980,29052,179923,411753,0.477007,0.162953,0.152348,0.012095
5,5,1WEJ,10413,25245,3225,4320,18695,160359,359414,0.556994,0.157428,0.172506,0.012020
6,6,1ADQ,8158,17863,2196,2669,18626,199187,383757,0.437990,0.089680,0.117900,0.006955
7,7,1FBI,7135,16884,2112,2853,15390,147163,402274,0.463613,0.114730,0.137232,0.007092
8,8,2YPV,13344,26078,3695,4218,26306,182158,407418,0.507261,0.143161,0.140462,0.010353
9,9,1OB1,12537,25546,3496,3809,26466,224613,383466,0.473702,0.113733,0.132094,0.009933


In [7]:
# We generate new MiniAbsolut Antigens
# We leave the high unchanged and adapt the weak and non-binders accordingly!
# Such an adjustment is possible (see df_res above).
# Changing the high is impossible to 15k sequences, and also it would be nice
# to keep the same size of high. 
# Don't forget later to account for the difference in %similar to w/n in high (40-60% range)

## Note: the sequences are represented symmetrically repertoire 1>2 and 2>1.

In [8]:
for ag in config.ANTIGENS:
    print(f"Working on {ag}")

    ## Prepare save directory, new ag
    base_p = Path(f"../data/MiniAbsolut")
    #SIM
    ag_new_sim = f"{ag}SIM"
    ag_new_sim_p = base_p / ag_new_sim
    ag_new_sim_p.mkdir(exist_ok=True)
    #DIF
    ag_new_dif = f"{ag}DIF"
    ag_new_dif_p = base_p / ag_new_dif
    ag_new_dif_p.mkdir(exist_ok=True)

    ## Read the pairs computed from CompAIRR
    df_pairs  = pd.read_csv(
                f"../data/MiniAbsolut/{ag}/AIRR/overlaps_d1_pairs.tsv",
                sep='\t',
            )
    df_pairs = df_pairs.rename(columns={"#repertoire_id_1": "repertoire_id_1"})

    ## Read the high sequences, to be used to find neighbours
    df_high_train = pd.read_csv(
        f"../data/MiniAbsolut/{ag}/high_train_15000.tsv",
        sep='\t',
    )

    ## Adapt the vs Weak
    # Filter for relevant pairs
    df_hw = df_pairs.query(f"repertoire_id_1 == '{ag}_high' and repertoire_id_2 == '{ag}_looserX'")
    df_hw_in_high = df_hw.loc[df_hw["junction_aa_1"].isin(df_high_train['Slide'])]

    df_weak_train = pd.read_csv(
        f"../data/MiniAbsolut/{ag}/looserX_train_15000.tsv",
        sep='\t',
    )
    df_weak_rest = pd.read_csv(
        f"../data/MiniAbsolut/{ag}/looserX_rest.tsv",
        sep='\t',
    )
    df_weak = pd.concat([df_weak_train, df_weak_rest])

    # Find the sequences in the weak that are similar to high
    df_weak_in_hw = df_weak.loc[df_weak["Slide"].isin(df_hw_in_high['junction_aa_2'])]

    # Generate sim
    df_weak_sim = df_weak_in_hw.copy()
    try:
        df_weak_train_sim = df_weak_sim.sample(15000, random_state=42)
    except ValueError:
        print("Not enough sequences in weak to sample 15000 for SIM, sampling all")
        df_weak_train_sim = df_weak_sim.sample(df_weak_sim.shape[0], random_state=42)
    df_weak_rest_sim = df_weak_sim.loc[~df_weak_sim["Slide"].isin(df_weak_train_sim["Slide"])]
    # Save
    df_weak_train_sim.to_csv(ag_new_sim_p / "looserX_train_15000.tsv", sep='\t', index=False)
    df_weak_rest_sim.to_csv(ag_new_sim_p / "looserX_rest.tsv", sep='\t', index=False)

    # Generate dif
    df_weak_dif = df_weak.loc[~df_weak["Slide"].isin(df_weak_in_hw["Slide"])]
    try:
        df_weak_train_dif = df_weak_dif.sample(15000, random_state=42)
    except ValueError:
        print("Not enough sequences in weak to sample 15000 for DIF, sampling all")
        df_weak_train_dif = df_weak_dif.sample(df_weak_dif.shape[0], random_state=42)
    df_weak_rest_dif = df_weak_dif.loc[~df_weak_dif["Slide"].isin(df_weak_train_dif["Slide"])]
    # Save
    df_weak_train_dif.to_csv(ag_new_dif_p / "looserX_train_15000.tsv", sep='\t', index=False)
    df_weak_rest_dif.to_csv(ag_new_dif_p / "looserX_rest.tsv", sep='\t', index=False)

    ## Adapt the vs Nonbinder
    # Filter for relevant pairs
    df_hn = df_pairs.query(f"repertoire_id_1 == '{ag}_high' and repertoire_id_2 == '{ag}_95low'")
    df_hn_in_high = df_hn.loc[df_hn["junction_aa_1"].isin(df_high_train['Slide'])]

    df_nonb_train = pd.read_csv(
        f"../data/MiniAbsolut/{ag}/95low_train_15000.tsv",
        sep='\t',
    )
    df_nonb_rest = pd.read_csv(
        f"../data/MiniAbsolut/{ag}/95low_rest.tsv",
        sep='\t',
    )
    df_nonb = pd.concat([df_nonb_train, df_nonb_rest])

    # Find the sequences in the nonbinder that are similar to high
    df_nonb_in_hn = df_nonb.loc[df_nonb["Slide"].isin(df_hn_in_high['junction_aa_2'])]

    # Generate sim
    df_nonb_sim = df_nonb_in_hn.copy()
    try:
        df_nonb_train_sim = df_nonb_sim.sample(15000, random_state=42)
    except ValueError:
        print("Not enough sequences in nonbinder to sample 15000 for SIM, sampling all")
        df_nonb_train_sim = df_nonb_sim.sample(df_nonb_sim.shape[0], random_state=42)
    df_nonb_rest_sim = df_nonb_sim.loc[~df_nonb_sim["Slide"].isin(df_nonb_train_sim["Slide"])]
    # Save
    df_nonb_train_sim.to_csv(ag_new_sim_p / "95low_train_15000.tsv", sep='\t', index=False)
    df_nonb_rest_sim.to_csv(ag_new_sim_p / "95low_rest.tsv", sep='\t', index=False)

    # Generate dif
    df_nonb_dif = df_nonb.loc[~df_nonb["Slide"].isin(df_nonb_in_hn["Slide"])]
    try:
        df_nonb_train_dif = df_nonb_dif.sample(15000, random_state=42)
    except ValueError:
        print("Not enough sequences in nonbinder to sample 15000 for DIF, sampling all")
        df_nonb_train_dif = df_nonb_dif.sample(df_nonb_dif.shape[0], random_state=42)
    df_nonb_rest_dif = df_nonb_dif.loc[~df_nonb_dif["Slide"].isin(df_nonb_train_dif["Slide"])]
    # Save
    df_nonb_train_dif.to_csv(ag_new_dif_p / "95low_train_15000.tsv", sep='\t', index=False)
    df_nonb_rest_dif.to_csv(ag_new_dif_p / "95low_rest.tsv", sep='\t', index=False)

    print(f"#high with similar in weak: {df_high_train['Slide'].isin(df_hw['junction_aa_1']).sum()}")
    print(f"#weak possible, with seq in hw: {df_weak_in_hw.shape[0]}")
    print(f"#high with similar in nonb: {df_high_train['Slide'].isin(df_hn['junction_aa_1']).sum()}")
    print(f"#nonb possible, with seq in hn: {df_nonb_in_hn.shape[0]}")
    print("\n")

    ## Copy high train to new antigen
    shutil.copy(
        f"../data/MiniAbsolut/{ag}/high_train_15000.tsv", 
        ag_new_sim_p / "high_train_15000.tsv"
    )
    shutil.copy(
        f"../data/MiniAbsolut/{ag}/high_train_15000.tsv", 
        ag_new_dif_p / "high_train_15000.tsv"
    )
    ## Copy all test to new antigen
    for f in Path(f"../data/MiniAbsolut/{ag}").glob("*test*.tsv"):
        shutil.copy(f, ag_new_sim_p / f.name)
        shutil.copy(f, ag_new_dif_p / f.name)

Working on 3VRL


Not enough sequences in nonbinder to sample 15000 for SIM, sampling all
#high with similar in weak: 8676
#weak possible, with seq in hw: 24197
#high with similar in nonb: 2036
#nonb possible, with seq in hn: 2567


Working on 1NSN
Not enough sequences in weak to sample 15000 for SIM, sampling all
Not enough sequences in nonbinder to sample 15000 for SIM, sampling all
#high with similar in weak: 6469
#weak possible, with seq in hw: 13213
#high with similar in nonb: 2518
#nonb possible, with seq in hn: 2845


Working on 3RAJ
Not enough sequences in nonbinder to sample 15000 for SIM, sampling all
#high with similar in weak: 7176
#weak possible, with seq in hw: 19646
#high with similar in nonb: 1489
#nonb possible, with seq in hn: 1875


Working on 5E94
Not enough sequences in nonbinder to sample 15000 for SIM, sampling all
#high with similar in weak: 9046
#weak possible, with seq in hw: 25667
#high with similar in nonb: 1459
#nonb possible, with seq in hn: 1751


Working on 1H0D
Not enoug